<a target="_blank" href="https://colab.research.google.com/github/rcpaffenroth/dac_raghu/blob/main/LunarLander.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Setup and libraries

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules

In [14]:
if IN_COLAB:
  ! apt-get install swig
  ! pip install stable-baselines3[extra] gymnasium[box2d] huggingface_sb3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.7 MB/s eta 0:00:00


In [45]:
import gymnasium as gym
import matplotlib.pylab as py
import numpy as np

import imageio
from stable_baselines3 import PPO
from huggingface_sb3 import load_from_hub

import pandas as pa

from IPython import display
from IPython.display import HTML
from base64 import b64encode
%matplotlib inline

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
# Make the environment
env = gym.make("LunarLander-v2", render_mode='rgb_array')

observation = env.reset()
# Note, there are warnings here that I should fix at some point.

# Train a model

In [35]:
class RandomModel(object):
  def __init__(self, env):
    self.env = env

  def predict(self, obs):
    return env.action_space.sample()

random_model =  RandomModel(env)

In [36]:
# This is an trained model that has a good architecture and loss function, but is not trained very much
trained_model = PPO("MlpPolicy", env)
trained_model.learn(total_timesteps=10000)

In [37]:
# checkpoint = load_from_hub(
#     repo_id="sb3/demo-hf-CartPole-v1",
#     filename="ppo-CartPole-v1.zip",
# )
checkpoint = load_from_hub(
    # repo_id="MalarzDawid/ppo-LunarLandar-v2",
    repo_id="sb3/a2c-LunarLander-v2",
    # filename="ppo-LunarLander-v2.zip",
    filename="a2c-LunarLander-v2.zip",
)

sota_model = PPO.load(checkpoint)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: 'bytes' object cannot be interpreted as an integer
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: 'bytes' object cannot be interpreted as an integer
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(


# Untrained model

In [38]:
# Make a movie of a trained agent
obs = env.reset()[0]
images = []
done = False
while not done:
   # This rendering mode puts an image into a numpy array
   images += [env.render()]
   action = random_model.predict(obs)
   obs, reward, done, trunc, info = env.step(action)
env.close()

In [39]:
# imageio is a nice library for taking a sequence of images and makeing a movie
name = 'tmp.mp4'
imageio.mimsave(name, images, fps=15)
mp4 = open(name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Trained model

In [40]:
# Make a movie of a trained agent
obs = env.reset()[0]
images = []
done = False
while not done:
   # This rendering mode puts an image into a numpy array
   images += [env.render()]
   action, _state = trained_model.predict(obs)
   obs, reward, done, trunc, info = env.step(action)
env.close()

In [41]:
# imageio is a nice library for taking a sequence of images and makeing a movie
name = 'tmp.mp4'
imageio.mimsave(name, images, fps=15)
mp4 = open(name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# SotA model

In [42]:
# Make a movie of a trained agent
obs = env.reset()[0]
images = []
done = False
while not done:
   # This rendering mode puts an image into a numpy array
   images += [env.render()]
   action, _state = sota_model.predict(obs)
   obs, reward, done, trunc, info = env.step(action)
env.close()

In [43]:
# imageio is a nice library for taking a sequence of images and makeing a movie
name = 'tmp.mp4'
imageio.mimsave(name, images, fps=15)
mp4 = open(name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)